# Demo notebook for slide 3.4

In [35]:
%matplotlib widget

In [36]:
import bmcs_utils
from ibvpy.tfunction import TimeFunction, TFSelector, TFCyclicSin
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

## Study #1 - simultaneously pull off and circularly slide 

In [37]:
material_params = dict(
    E_T=1, gamma_T=100, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

In [38]:
se = SlideExplorer(n_steps=2000, k_max=50)
se.slide_model.trait_set(**material_params)
n_cycles=3
se.tf_s_x = TFCyclicSin(number_of_cycles = n_cycles, t_max = 1)
se.tf_s_y = TFCyclicSin(number_of_cycles = n_cycles, t_max = 1, phase_shift=0.25)
#se.tf_w = TFCyclicSin(number_of_cycles = n_cycles, t_max = 1)
se.trait_set(s_x_1 = 4.1, s_y_1 = 8.1, w_1 = 3);
se.run()
se.energy_dissipation.show_iter = False
se.interact()

# Study #2 - push first and then circularly slide

In [39]:
material_params = dict(
    E_T=1, gamma_T=100, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=1.0 # 0.5
)

In [34]:
se = SlideExplorer(n_steps=2000, k_max=50)
se.slide_model.trait_set(**material_params)
n_cycles=3
se.tf_s_x = TFCyclicSin(number_of_cycles = n_cycles, t_max = 1)
#se.tf_s_y = TFCyclicSin(number_of_cycles = n_cycles, t_max = 1, phase_shift=0.25)
#se.tf_w = TFCyclicSin(number_of_cycles = n_cycles, t_max = 1)
se.slide_model.trait_set(**material_params)
se.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = -25);
se.run()
se.trait_set(s_x_1 = 6.1, s_y_1 = 0, w_1 = -25);
se.run()
se.interact()

In [20]:
import numpy as np
material_params = dict(
    E_T=1, gamma_T=0, K_T=1, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.5
)
eta_range = np.linspace(0,1,5)
se_T_list = []
se_N_list = []
#for m in [1e-9, 0.05]:
for eta in eta_range:
    material_params['eta'] = eta
    se1 = SlideExplorer(n_steps=100, k_max=50)
    se1.slide_model.trait_set(**material_params)
#     se1.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 0.8);
#     se1.run()
    se1.trait_set(s_x_1 = 20, s_y_1 = 0, w_1 = 0);
    se1.run()
    se_T_list.append(se1)
    se2 = SlideExplorer(n_steps=200, k_max=50)
    se2.slide_model.trait_set(**material_params)
    se2.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 20);
    se2.run()
    se_N_list.append(se2)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pylab as plt
fig, (ax_T, ax_N) = plt.subplots(1,2, tight_layout=True, figsize=(10,4))
fig.canvas.header_visible=False
ax_T.plot([0,1],[0,1],'--',color='black')
ax_N.plot([0,1],[0,1],'--',color='black')
for se_T, se_N in zip(se_T_list, se_N_list):
    s_x_pi_, s_y_pi_, w_pi_, z_, alpha_x_, alpha_y_, omega_T_, omega_N_ = se_N.Eps_arr.T
    #ax_T.plot(omega_T_,omega_N_)
    se_T.inel_state_evolution.plot_omega_NT(ax_T)
    se_N.inel_state_evolution.plot_omega_NT(ax_N)

In [ ]:
fig.savefig('myfig.pdf')

In [ ]:
se_T_list[1].inel_state_evolution.interact()

In [ ]:
import sympy as sp
S_N, S_T, eta = sp.symbols('S_N, S_T, eta')
Y_N, Y_T = sp.symbols('Y_N, Y_T')
omega_N, omega_T = sp.symbols('omega_N, omega_T')
c_N, c_T = sp.symbols('c_N, c_T')

phi_N_ext = ( # (1-omega_N)**c_N * (
    (Y_N**2 + eta * (Y_T * Y_N)) /
    (2*(S_N - eta * (S_N - sp.sqrt(S_N * S_T))))
)
phi_T_ext = ( # (1-omega_T)**c_T * (
    (Y_T**2 + eta * (Y_T * Y_N)) /
    (2*(S_T - eta * (S_T - sp.sqrt(S_N * S_T))))
)
phi_ext = phi_N_ext + phi_T_ext

In [ ]:
sp.simplify(phi_ext.diff(Y_N).subs(eta,1))

In [ ]:
sp.simplify(phi_ext.diff(Y_T).subs(eta,1))

In [ ]:
sp.simplify(phi_ext)

In [ ]:
import sympy as sp
E_, K_ = sp.symbols('E, K')
get_EK = sp.lambdify( (E_,K_) , E_*K_ / (E_+K_) - E_ )
_, ax_EK = plt.subplots(1,1)
K_r = np.linspace(1,100,100)
ax_EK.plot(K_r, get_EK(1,K_r))

# Debugging support

You can access the values of the state variables and of the the associated 
thermodynamic forces by accessing the `Sig_t` and `Eps_t` arrays.
Each row represents the time step and each column a state variable.
The order of state variables corresponds to the order in the symbolic 
object `se.slide_model.symb`

In [ ]:
se.Eps_t[-1,:], se.Sig_t[-1,:]

In [ ]:
se.Sig_arr[-1,:]